In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xgboost as xgb

from src.constant import PROCESSED_DATA_DIR

plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["axes.grid"] = True
plt.rcParams["grid.alpha"] = 0.2
plt.rcParams["grid.color"] = "#cccccc"
plt.rcParams["axes.xmargin"] = 0

In [4]:
evaluations_df = pd.read_parquet(PROCESSED_DATA_DIR / "evaluations.parquet")
solvers_df = pd.read_parquet(PROCESSED_DATA_DIR / "solvers.parquet")
instances_df = pd.read_parquet(PROCESSED_DATA_DIR / "instances.parquet")

df = pd.merge(evaluations_df, solvers_df, left_on="solver_id", right_on="id").drop(columns=["id"])
df = pd.merge(df, instances_df, left_on="instance_id", right_on="id").drop(columns=["id"])
df

,solver_id,instance_id,generator,cost,ASCENT_CANDIDATES,BACKBONE_TRIALS,BACKTRACKING,CANDIDATE_SET_TYPE,EXTRA_CANDIDATES,EXTRA_CANDIDATE_SET_TYPE,...,mst_dists_span,mst_dists_coef_of_var,mst_dists_sum,nnds_min,nnds_median,nnds_mean,nnds_max,nnds_sd,nnds_span,nnds_coef_of_var
0,1251473931473582278,TSP/TRAIN/cluster_netgen/000.tsp,cluster_netgen,4.74,0.95,1.0,0.0,0.0,0.2,0.0,...,0.488488,1.790135,0.000225,0.000715,0.011143,0.014421,0.094965,0.012746,0.094250,0.883825
1,2289112522627003788,TSP/TRAIN/cluster_netgen/000.tsp,cluster_netgen,0.02,0.15,0.0,1.0,1.0,0.4,0.0,...,0.488488,1.790135,0.000225,0.000715,0.011143,0.014421,0.094965,0.012746,0.094250,0.883825
2,960932965817811220,TSP/TRAIN/cluster_netgen/000.tsp,cluster_netgen,3.72,0.20,0.0,1.0,2.0,0.9,0.0,...,0.488488,1.790135,0.000225,0.000715,0.011143,0.014421,0.094965,0.012746,0.094250,0.883825
3,39012066323493184,TSP/TRAIN/cluster_netgen/000.tsp,cluster_netgen,1.52,0.60,1.0,1.0,2.0,0.7,0.0,...,0.488488,1.790135,0.000225,0.000715,0.011143,0.014421,0.094965,0.012746,0.094250,0.883825
4,494182449327999052,TSP/TRAIN/cluster_netgen/000.tsp,cluster_netgen,84.44,0.90,1.0,1.0,3.0,0.3,0.0,...,0.488488,1.790135,0.000225,0.000715,0.011143,0.014421,0.094965,0.012746,0.094250,0.883825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1286196444435323941,TSP/TRAIN/grid/019.tsp,grid,208.32,0.45,0.0,0.0,0.0,0.2,0.0,...,0.093952,0.462330,0.000294,0.001335,0.029728,0.032519,0.095287,0.017339,0.093952,0.533201
99996,1435531534300921454,TSP/TRAIN/grid/019.tsp,grid,300.00,0.20,1.0,0.0,3.0,0.6,0.0,...,0.093952,0.462330,0.000294,0.001335,0.029728,0.032519,0.095287,0.017339,0.093952,0.533201
99997,27607668447685341,TSP/TRAIN/grid/019.tsp,grid,3.21,0.95,1.0,1.0,3.0,0.9,0.0,...,0.093952,0.462330,0.000294,0.001335,0.029728,0.032519,0.095287,0.017339,0.093952,0.533201
99998,2245205590089179674,TSP/TRAIN/grid/019.tsp,grid,17.23,0.65,0.0,0.0,1.0,0.5,0.0,...,0.093952,0.462330,0.000294,0.001335,0.029728,0.032519,0.095287,0.017339,0.093952,0.533201


In [5]:
from src.evaluation import evaluate_model_with_cross_validation
from src.split import permutate_df_by_cost_decreasing
from src.split import get_n_splits
from src.configspace import XGB_AFT_CONFIGSPACE

In [6]:
train_idx, test_idx = get_n_splits(df, n=2, instance_number=5, solver_number=100, random_state=0)[0]

df_train = df.loc[train_idx]
df_test = df.loc[test_idx]

not_train_cols = ["solver_id", "instance_id", "generator", "cost"]

X_train = df_train.drop(columns=not_train_cols)
y_train = df_train["cost"].to_numpy()

X_test = df_test.drop(columns=not_train_cols)
y_test = df_test["cost"].to_numpy()
y_test_not_censored = y_test.copy()

const_cut_off = 1.0

cut_off_train = np.full(X_train.shape[0], const_cut_off)
cut_off_test = np.full(X_test.shape[0], const_cut_off)

y_train = np.clip(y_train, 0, cut_off_train)
y_test = np.clip(y_test, 0, cut_off_test)

dtrain = xgb.DMatrix(X_train)

In [28]:
# ...existing code...
# from the docs
# objective='survival:cox' - Cox regression for right censored survival time data (negative values are considered right censored). Note that predictions are returned on the hazard ratio scale (i.e., as HR = exp(marginal_prediction) in the proportional hazard function h(t) = h0(t) * HR).
# eval_metric='cox-nloglik' - negative partial log-likelihood for Cox proportional hazards regression

params = {
    'objective': 'survival:cox',
    'eval_metric': 'cox-nloglik',
    'colsample_bytree': 1.0,
    'gamma': 0.0,
    'learning_rate': 0.1,
    'max_depth': 6,
    'min_child_weight': 1,
    'reg_alpha': 0.001,
    'reg_lambda': 0.001,
    'seed': 0,
    'subsample': 1.0,
}

dtrain = xgb.DMatrix(X_train)

y_cox = np.where(y_train < cut_off_train, y_train, -cut_off_train)

dtrain.set_label(y_cox)

bst = xgb.train(
    params,
    dtrain,
    num_boost_round=100,
    evals=[(dtrain, "train")],
    verbose_eval=False,
)

hazard_ratios = bst.predict(dtrain)
hazard_ratios.shape

(250,)

In [ ]:
# Add a new cell with this code:

# Step 1: Create a time grid for survival function evaluation
# You need to define the time points at which you want to evaluate survival probabilities
time_points = np.linspace(0, const_cut_off, 100)  # 100 points from 0 to cutoff

# Step 2: Estimate the baseline hazard function
def estimate_baseline_hazard(model, X, y, censoring):
    """
    Estimate the baseline cumulative hazard function for Cox model
    
    Args:
        model: Trained XGBoost Cox model
        X: Feature data
        y: Time data
        censoring: Boolean array indicating which samples are censored
    
    Returns:
        Tuple of (time_points, baseline_hazard)
    """
    # Get the hazard ratios
    hazard_ratios = np.exp(model.predict(xgb.DMatrix(X)))
    
    # Sort data by time
    order = np.argsort(y)
    times = y[order]
    hazard_ratios = hazard_ratios[order]
    censoring = censoring[order]
    
    # Unique failure times and baseline hazard contributions
    unique_times = []
    baseline_hazard = []
    
    # Keep track of the risk set
    at_risk = np.sum(hazard_ratios)
    
    for i in range(len(times)):
        if not censoring[i]:  # Only consider uncensored events
            if i == 0 or times[i] > times[i-1]:
                unique_times.append(times[i])
                baseline_hazard.append(1.0 / at_risk)
            else:
                # Add to the previous baseline hazard for tied event times
                baseline_hazard[-1] += 1.0 / at_risk
        
        at_risk -= hazard_ratios[i]
    
    # Convert to cumulative baseline hazard
    baseline_cum_hazard = np.cumsum(baseline_hazard)
    
    return np.array(unique_times), np.array(baseline_cum_hazard)

# Determine which samples are censored (reaching the cutoff)
censored_train = y_train >= cut_off_train

# Estimate the baseline hazard
unique_times, baseline_cum_hazard = estimate_baseline_hazard(
    bst, X_train, y_train, censored_train
)

# Step 3: Calculate survival probabilities for each sample
def predict_survival_function(model, X, unique_times, baseline_cum_hazard):
    """
    Predict survival function for each sample
    
    Args:
        model: Trained XGBoost Cox model
        X: Feature data
        unique_times: Time points from baseline hazard estimation
        baseline_cum_hazard: Cumulative baseline hazard values
    
    Returns:
        Dictionary mapping sample indices to (times, survival_probs)
    """
    # Get hazard ratios for each sample
    dtest = xgb.DMatrix(X)
    hazard_ratios = np.exp(model.predict(dtest))
    
    survival_functions = {}
    
    for i in range(len(X)):
        # S(t) = exp(-H₀(t) × HR)
        survival_probs = np.exp(-baseline_cum_hazard * hazard_ratios[i])
        survival_functions[i] = (unique_times, survival_probs)
    
    return survival_functions

# Get survival functions for training samples
survival_functions = predict_survival_function(
    bst, X_train[:5],  # Just predict for first 5 samples as example
    unique_times, 
    baseline_cum_hazard
)

# Plot survival curves for a few examples
plt.figure(figsize=(10, 6))
for i, (times, surv) in list(survival_functions.items())[:5]:
    plt.step(times, surv, where='post', label=f'Sample {i}')

plt.xlabel('Time')
plt.ylabel('Survival Probability')
plt.title('Estimated Survival Functions')
plt.legend()
plt.grid(True)
plt.ylim(0, 1)
plt.show()